In [2]:
import sympy
sympy.init_printing(use_latex="mathjax", fontsize='16pt')

w, x, y, z = sympy.symbols("q.w, q.x, q.y, q.z")
b_x, b_y, b_z = sympy.symbols("b.x, b.y, b.z")
w_x, w_y, w_z = sympy.symbols("w.x,w.y,w.z")

update_matrix = sympy.Matrix([
    [1, -(w_x-b_x), -(w_y-b_y), -(w_z-b_z)],
    [(w_x-b_x), 1, (w_z-b_z), -(w_y-b_y)],
    [(w_y-b_y), -(w_z-b_z), 1, (w_x-b_x)],
    [(w_z-b_z), (w_y-b_y), -(w_x-b_x), 1],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
]) * sympy.Matrix([ w,  x, y, z ]) + sympy.Matrix([
    [0],
    [0],
    [0],
    [0],
    [b_x],
    [b_y],
    [b_z],
])


update_matrix

⎡ q.w + q.x⋅(b.x - w.x) + q.y⋅(b.y - w.y) + q.z⋅(b.z - w.z) ⎤
⎢                                                           ⎥
⎢q.w⋅(-b.x + w.x) + q.x + q.y⋅(-b.z + w.z) + q.z⋅(b.y - w.y)⎥
⎢                                                           ⎥
⎢q.w⋅(-b.y + w.y) + q.x⋅(b.z - w.z) + q.y + q.z⋅(-b.x + w.x)⎥
⎢                                                           ⎥
⎢q.w⋅(-b.z + w.z) + q.x⋅(-b.y + w.y) + q.y⋅(b.x - w.x) + q.z⎥
⎢                                                           ⎥
⎢                            b.x                            ⎥
⎢                                                           ⎥
⎢                            b.y                            ⎥
⎢                                                           ⎥
⎣                            b.z                            ⎦

In [3]:
FxX = update_matrix.jacobian(sympy.Matrix([w, x, y, z, b_x, b_y, b_z ]))
FxX

⎡    1       b.x - w.x   b.y - w.y   b.z - w.z   q.x   q.y   q.z ⎤
⎢                                                                ⎥
⎢-b.x + w.x      1       -b.z + w.z  b.y - w.y   -q.w  q.z   -q.y⎥
⎢                                                                ⎥
⎢-b.y + w.y  b.z - w.z       1       -b.x + w.x  -q.z  -q.w  q.x ⎥
⎢                                                                ⎥
⎢-b.z + w.z  -b.y + w.y  b.x - w.x       1       q.y   -q.x  -q.w⎥
⎢                                                                ⎥
⎢    0           0           0           0        1     0     0  ⎥
⎢                                                                ⎥
⎢    0           0           0           0        0     1     0  ⎥
⎢                                                                ⎥
⎣    0           0           0           0        0     0     1  ⎦

In [4]:
FxU=update_matrix.jacobian(sympy.Matrix([ w_x, w_y, w_z ]))
FxU

⎡-q.x  -q.y  -q.z⎤
⎢                ⎥
⎢q.w   -q.z  q.y ⎥
⎢                ⎥
⎢q.z   q.w   -q.x⎥
⎢                ⎥
⎢-q.y  q.x   q.w ⎥
⎢                ⎥
⎢ 0     0     0  ⎥
⎢                ⎥
⎢ 0     0     0  ⎥
⎢                ⎥
⎣ 0     0     0  ⎦

In [5]:
P = sympy.MatrixSymbol('P', 7, 7)
W = sympy.MatrixSymbol('W', 3, 3)

P_p_predicted = FxX * P * FxX.transpose() + FxU * W * FxU.transpose()
P_p_predicted

⎡-q.x  -q.y  -q.z⎤                                       ⎡    1       b.x - w. ↪
⎢                ⎥                                       ⎢                     ↪
⎢q.w   -q.z  q.y ⎥                                       ⎢-b.x + w.x      1    ↪
⎢                ⎥                                       ⎢                     ↪
⎢q.z   q.w   -q.x⎥   ⎡-q.x  q.w   q.z   -q.y  0  0  0⎤   ⎢-b.y + w.y  b.z - w. ↪
⎢                ⎥   ⎢                               ⎥   ⎢                     ↪
⎢-q.y  q.x   q.w ⎥⋅W⋅⎢-q.y  -q.z  q.w   q.x   0  0  0⎥ + ⎢-b.z + w.z  -b.y + w ↪
⎢                ⎥   ⎢                               ⎥   ⎢                     ↪
⎢ 0     0     0  ⎥   ⎣-q.z  q.y   -q.x  q.w   0  0  0⎦   ⎢    0           0    ↪
⎢                ⎥                                       ⎢                     ↪
⎢ 0     0     0  ⎥                                       ⎢    0           0    ↪
⎢                ⎥                                       ⎢                     ↪
⎣ 0     0     0  ⎦          

In [6]:
sympy.cse(P_p_predicted)

⎛                                                                              ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎜[(x₀, -q.x), (x₁, -q.y), (x₂, -q.z), (x₃, b.x - w.x), (x₄, b.y - w.y), (x₅, b ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎜                                                                              ↪
⎝                           

In [7]:
# sympy.ccode(P_p_predicted[0], strict=False, contract=True)
import sympy.codegen
import sympy.codegen.rewriting


p_pred = sympy.MatrixSymbol('p_pred', 7, 7)
x_pred = sympy.MatrixSymbol('x_pred', 7, 1)
# p_pred[0,0]
# sympy.ccode(update_matrix, assign_to=x_pred, strict=False)
with open("./generated/predict_covar.cpp", "w") as f:
    f.write(sympy.ccode(P_p_predicted, assign_to=p_pred))


In [8]:
with open("./generated/predict.cpp", "w") as f:
    f.write(sympy.ccode(update_matrix, assign_to=x_pred, strict=False))